In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.visualization.visualization import TrajectoryPlotter as viz
import geopandas as gpd

pdf = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/starkey.csv')
starkey = PTRAILDataFrame(data_set=pdf,
                          latitude='lat',
                          longitude='lon',
                          datetime='DateTime',
                          traj_id='Id')
starkey.head(5)

lat         lon  StarkeyTime    GMDate  \
traj_id   DateTime                                                            
880109D01 1995-04-13 13:40:06  45.239682 -118.533204    229902006  21:40:06   
          1995-04-15 12:16:15  45.250521 -118.530438    230069775  20:16:15   
          1995-04-15 21:39:38  45.247943 -118.541455    230103578  05:39:38   
          1995-04-16 03:32:14  45.247429 -118.539530    230124734  11:32:14   
          1995-04-16 04:08:28  45.247117 -118.542579    230126908  12:08:28   

                                 GMTime   LocDate   LocTime  RadNum Species  \
traj_id   DateTime                                                            
880109D01 1995-04-13 13:40:06  19950413  19950413  13:40:06     409       D   
          1995-04-15 12:16:15  19950415  19950415  12:16:15     409       D   
          1995-04-15 21:39:38  19950416  19950415  21:39:38     409       D   
          1995-04-16 03:32:14  19950416  19950416  03:32:14     409       D   
          1995-04-16 04:08:28  19950416  19950416  04:08:28     409       D   

                                 UTME     UTMN  Year  Grensunr  Grensuns  \
traj_id   DateTime                                                         
880109D01 1995-04-13 13:40:06  379662  5010734    95  13:13:00  02:39:00   
          1995-04-15 12:16:15  379895  5011927    95  13:09:00  02:41:00   
          1995-04-15 21:39:38  379039  5011656    95  13:07:00  02:43:00   
          1995-04-16 03:32:14  379188  5011581    95  13:07:00  02:43:00   
          1995-04-16 04:08:28  378938  5011567    95  13:07:00  02:43:00   

                               Obswt  
traj_id   DateTime                    
880109D01 1995-04-13 13:40:06   1.47  
          1995-04-15 12:16:15   1.59  
          1995-04-15 21:39:38   1.34  
          1995-04-16 03:32:14   1.50  
          1995-04-16 04:08:28   1.34

In [2]:
starkey_habitat = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/starkey_habitat.csv')
starkey_habitat.head()

,lat,lon,SoilDpth,PerSlope,SINAspct,COSAspct,Convex3,DistCWat,Canopy,Elev,DistEWat,EcoGener,DistOPEN,DistRSTR,DistCLSD,DistEFnc,CowPast,ForgProd,DistEdge
0,45.272378,-118.610193,14,5,-0.81,0.59,500.63,218,3,1389,212,GB,127,2271,30,0,SMITH-BALLY,363,0
1,45.272648,-118.610201,14,5,-0.71,0.71,500.21,228,3,1388,218,GB,150,2293,30,0,SMITH-BALLY,363,0
2,45.272918,-118.610208,14,4,-0.89,0.45,499.93,242,3,1387,228,GB,170,2315,30,0,SMITH-BALLY,363,0
3,45.270224,-118.609750,14,8,-0.98,0.21,500.68,201,5,1393,201,GB,30,2081,134,0,SMITH-BALLY,363,0
4,45.270494,-118.609757,14,7,-0.97,0.22,501.22,190,5,1393,190,GB,30,2101,108,0,SMITH-BALLY,363,0


In [3]:
viz.plot_folium_traj(starkey, opacity=1, weight=2)